In [7]:
import numpy as np
import mne
import pandas as pd
import sys

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [50]:

edf = mne.io.read_raw_edf('chb01_04.edf')
header = ','.join(edf.ch_names)


Extracting EDF parameters from F:\PY\Pro\pro\BTP_Seizure_Pred\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-50-6b5873c81120>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf = mne.io.read_raw_edf('chb01_04.edf')


In [9]:
#edf_f = edf.filter(l_freq = 4 , h_freq = 14)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 423 samples (1.652 sec)



In [ ]:
#np.savetxt('your_csv_file.csv', edf.get_data().T, delimiter=',', header=header)

In [35]:
#functions for calc of stat features
def mean_variance(df):
    variance_vals = np.var(df)
    return np.mean(variance_vals)

In [51]:
seizure_start = 1467
seizure_end = 1494
#window function to work over data and calculate features
def add_feature(edf_file,start,duration):
    d = edf_file[:,256*start:256*(start+duration)]
    row_to_add = []
    mean_var = mean_variance(d[0])
    
    if start in range(seizure_start,(seizure_end,-1)):
        t = 1
    else:
        t = 0
    row_to_add.append(mean_var)
    row_to_add.append(start)
    row_to_add.append(t)
    return row_to_add

In [70]:
df = pd.DataFrame() #columns = ['mean_var','start','target']

In [71]:
#driver code
start = temp = 1397
duration = 10
end = 1574
while temp<end:
    row = add_feature(edf,temp,duration)
    df=df.append(pd.Series(row),ignore_index=True)
    temp = temp + duration

In [72]:
df

,0,1,2
0,1.045350e-09,1397.0,0.0
1,1.089297e-09,1407.0,0.0
2,1.245100e-09,1417.0,0.0
3,1.313426e-09,1427.0,0.0
4,1.010236e-09,1437.0,0.0
5,1.088562e-09,1447.0,0.0
6,3.048230e-09,1457.0,0.0
7,1.247817e-08,1467.0,1.0
8,1.114856e-08,1477.0,1.0
9,1.315812e-08,1487.0,1.0


In [32]:
df_f = pd.DataFrame()
start = temp = 2956
duration = 10
end = 3076
while temp<end:
    row = add_feature(edf_f,temp,duration)
    df_f=df_f.append(pd.Series(row),ignore_index=True)
    temp = temp + duration

In [33]:
df_f

,0,1
0,2.861505e-10,2956.0
1,3.494730e-10,2966.0
2,4.145274e-10,2976.0
3,3.188420e-10,2986.0
4,1.965291e-09,2996.0
5,2.459066e-09,3006.0
6,2.538410e-09,3016.0
7,1.361507e-09,3026.0
8,9.248479e-10,3036.0
9,9.684857e-10,3046.0


In [6]:
f = 2095
print(round(f,-1))

2100
